In [4]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from collections import Counter
import warnings
#from __future__ import division
import pandas as pd
import numpy as np
#import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, nltk, warnings
import matplotlib.cm as cm
import itertools
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from wordcloud import WordCloud, STOPWORDS
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from IPython.display import display, HTML
#import plotly.plotly as py
#import plotly.graph_objs as go
#from plotly.offline import init_notebook_mode,iplot
#init_notebook_mode(connected=True)
#warnings.filterwarnings("ignore")
#plt.rcParams["patch.force_edgecolor"] = True
#plt.style.use('fivethirtyeight')
#mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
%matplotlib inline
warnings.filterwarnings('ignore')

In [5]:
df_train=pd.read_csv('train.csv', header=0)
df_test=pd.read_csv('test.csv', header=0)
submission=pd.read_csv('submission.csv', header=0)

In [6]:
df_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [7]:
df_train['Segmentation'].value_counts()

D    2268
A    1972
C    1970
B    1858
Name: Segmentation, dtype: int64

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [9]:
x=df_train[df_train['Work_Experience']==0]

In [10]:
x[x['Profession']=='Artist']

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
5,461319,Male,Yes,56,No,Artist,0.0,Average,2.0,Cat_6,C
19,464291,Female,Yes,79,Yes,Artist,0.0,High,1.0,Cat_6,C
28,465882,Male,No,35,Yes,Artist,0.0,Low,1.0,Cat_6,B
30,463331,Female,Yes,45,Yes,Artist,0.0,Low,3.0,Cat_6,C
46,465729,Male,Yes,41,Yes,Artist,0.0,Average,4.0,Cat_4,B
...,...,...,...,...,...,...,...,...,...,...,...
8024,467153,Female,Yes,51,Yes,Artist,0.0,Low,NaN,Cat_6,A
8050,466004,Male,Yes,52,Yes,Artist,0.0,Average,6.0,Cat_4,C
8060,463613,Female,Yes,48,Yes,Artist,0.0,Average,6.0,Cat_6,A
8061,465231,Male,Yes,65,No,Artist,0.0,Average,2.0,Cat_6,C


In [11]:
map_gender={"Male" : 0, "Female": 1}
df_train['Gender'] = df_train['Gender'].map(map_gender)
df_test['Gender'] = df_test['Gender'].map(map_gender)


In [12]:
df_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,0,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,1,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,1,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,0,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,1,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [13]:
df_train.sort_values(["Age"], axis=0, ascending=[True], inplace=True) 
df_test.sort_values(["Age"], axis=0, ascending=[True], inplace=True) 

In [14]:
map_married={"No" : 0, "Yes": 1}
df_train['Ever_Married'] = df_train['Ever_Married'].map(map_married)
df_test['Ever_Married'] = df_test['Ever_Married'].map(map_married)

In [15]:
df_train["Ever_Married"].fillna( method ='ffill', inplace = True) 
df_test["Ever_Married"].fillna( method ='ffill', inplace = True) 

In [16]:
df_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
4321,461253,1,0.0,18,No,Healthcare,0.0,Low,5.0,Cat_4,D
3992,466750,1,0.0,18,NaN,Healthcare,0.0,Low,3.0,Cat_3,D
1155,466814,0,0.0,18,No,Healthcare,0.0,Low,3.0,Cat_6,D
1159,467474,0,0.0,18,No,Healthcare,3.0,Low,5.0,Cat_6,D
5827,462764,0,0.0,18,No,Healthcare,NaN,Low,4.0,Cat_6,D


In [17]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8068 entries, 4321 to 7262
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   int64  
 2   Ever_Married     8068 non-null   float64
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(3), int64(3), object(5)
memory usage: 756.4+ KB


In [18]:
df_train['Spending_Score'].value_counts()

Low        4878
Average    1974
High       1216
Name: Spending_Score, dtype: int64

In [19]:
map_score={"Low" : 1, "Average": 2,"High":3}
df_train['Spending_Score'] = df_train['Spending_Score'].map(map_score)
df_test['Spending_Score'] = df_test['Spending_Score'].map(map_score)

In [20]:
df_train.sort_values(["Spending_Score"], axis=0, ascending=[True], inplace=True) 
df_test.sort_values(["Spending_Score"], axis=0, ascending=[True], inplace=True) 

In [21]:
#df_train["Graduated"].fillna( method ='ffill', inplace = True) 
#df_test["Graduated"].fillna( method ='ffill', inplace = True) 

In [22]:
df_train

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
4321,461253,1,0.0,18,No,Healthcare,0.0,1,5.0,Cat_4,D
7458,463312,0,1.0,40,No,Engineer,3.0,1,1.0,Cat_3,A
5865,463348,1,1.0,40,Yes,Doctor,1.0,1,2.0,Cat_6,A
3321,466806,0,1.0,40,Yes,Healthcare,0.0,1,1.0,Cat_6,D
2352,467070,0,1.0,40,No,Executive,5.0,1,4.0,Cat_6,D
...,...,...,...,...,...,...,...,...,...,...,...
2111,465416,0,1.0,55,No,Engineer,1.0,3,4.0,Cat_6,A
4130,463722,0,1.0,55,Yes,Executive,9.0,3,2.0,Cat_6,C
1116,462068,0,1.0,55,Yes,Executive,1.0,3,3.0,Cat_6,C
5121,467654,1,1.0,56,Yes,Artist,NaN,3,4.0,Cat_6,C


In [23]:
map_garduate={"No" : 0, "Yes": 1}
df_train['Graduated'] = df_train['Graduated'].map(map_garduate)
df_test['Graduated'] = df_test['Graduated'].map(map_garduate)

In [24]:
df_train['Profession'].value_counts()

Artist           2516
Healthcare       1332
Entertainment     949
Engineer          699
Doctor            688
Lawyer            623
Executive         599
Marketing         292
Homemaker         246
Name: Profession, dtype: int64

In [25]:
map_profession={"Healthcare" : 1, "Engineer": 2,"Doctor":3,"Healthcare":4,"Executive":5}
df_train['Profession'] = df_train['Profession'].map(map_profession)
df_test['Profession'] = df_test['Profession'].map(map_profession)


In [26]:
df_train["Var_1"].value_counts()

Cat_6    5238
Cat_4    1089
Cat_3     822
Cat_2     422
Cat_7     203
Cat_1     133
Cat_5      85
Name: Var_1, dtype: int64

In [27]:
map_var={"Cat_1" : 1, "Cat_2": 2,"Cat_3":3,"Cat_4":4,"Cat_5":5,"Cat_6":6,"Cat_7":7}
df_train['Var_1'] = df_train['Var_1'].map(map_var)
df_test['Var_1'] = df_test['Var_1'].map(map_var)

In [28]:
df_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
4321,461253,1,0.0,18,0.0,4.0,0.0,1,5.0,4.0,D
7458,463312,0,1.0,40,0.0,2.0,3.0,1,1.0,3.0,A
5865,463348,1,1.0,40,1.0,3.0,1.0,1,2.0,6.0,A
3321,466806,0,1.0,40,1.0,4.0,0.0,1,1.0,6.0,D
2352,467070,0,1.0,40,0.0,5.0,5.0,1,4.0,6.0,D


In [29]:
df_train.corr()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
ID,1.000000,-0.006155,0.019420,-0.005055,-0.011544,-0.003726,-0.030688,0.005773,0.011801,-0.001116
Gender,-0.006155,1.000000,-0.114187,-0.030216,0.034109,-0.442594,0.059878,-0.061432,-0.055911,-0.030861
Ever_Married,0.019420,-0.114187,1.000000,0.568912,0.198102,0.025516,-0.095459,0.604278,-0.078956,0.084025
Age,-0.005055,-0.030216,0.568912,1.000000,0.237309,0.018807,-0.190789,0.415485,-0.280517,0.168308
Graduated,-0.011544,0.034109,0.198102,0.237309,1.000000,-0.029852,0.031374,0.103468,-0.231260,0.126698
Profession,-0.003726,-0.442594,0.025516,0.018807,-0.029852,1.000000,-0.021655,0.259112,0.156451,0.069117
Work_Experience,-0.030688,0.059878,-0.095459,-0.190789,0.031374,-0.021655,1.000000,-0.075194,-0.063234,0.023697
Spending_Score,0.005773,-0.061432,0.604278,0.415485,0.103468,0.259112,-0.075194,1.000000,0.094092,0.079258
Family_Size,0.011801,-0.055911,-0.078956,-0.280517,-0.231260,0.156451,-0.063234,0.094092,1.000000,-0.146823
Var_1,-0.001116,-0.030861,0.084025,0.168308,0.126698,0.069117,0.023697,0.079258,-0.146823,1.000000


In [30]:
df_train.sort_values(["Age"], axis=0, ascending=[True], inplace=True) 
df_test.sort_values(["Age"], axis=0, ascending=[True], inplace=True) 

In [31]:
df_train["Graduated"].fillna( method ='ffill', inplace = True) 
df_test["Graduated"].fillna( method ='ffill', inplace = True) 

In [32]:
df_train.sort_values(["Gender"], axis=0, ascending=[True], inplace=True) 
df_test.sort_values(["Gender"], axis=0, ascending=[True], inplace=True)

In [33]:
df_train["Profession"].fillna( method ='ffill', inplace = True) 
df_train["Profession"].fillna( method ='bfill', inplace = True) 
df_test["Profession"].fillna( method ='ffill', inplace = True)
df_test["Profession"].fillna( method ='bfill', inplace = True)

In [34]:
df_train.sort_values(["Age"], axis=0, ascending=[True], inplace=True) 
df_test.sort_values(["Age"], axis=0, ascending=[True], inplace=True) 

In [35]:
df_train["Work_Experience"].fillna( method ='ffill', inplace = True) 
df_test["Work_Experience"].fillna( method ='ffill', inplace = True)

In [36]:
df_train["Family_Size"].fillna( method ='ffill', inplace = True) 
df_test["Family_Size"].fillna( method ='ffill', inplace = True)
df_test["Family_Size"].fillna( method ='bfill', inplace = True)

In [37]:
df_train["Var_1"].fillna( method ='ffill', inplace = True) 
df_test["Var_1"].fillna( method ='ffill', inplace = True)

In [38]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8068 entries, 4321 to 4179
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   int64  
 2   Ever_Married     8068 non-null   float64
 3   Age              8068 non-null   int64  
 4   Graduated        8068 non-null   float64
 5   Profession       8068 non-null   float64
 6   Work_Experience  8068 non-null   float64
 7   Spending_Score   8068 non-null   int64  
 8   Family_Size      8068 non-null   float64
 9   Var_1            8068 non-null   float64
 10  Segmentation     8068 non-null   object 
dtypes: float64(6), int64(4), object(1)
memory usage: 756.4+ KB


In [39]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2627 entries, 1127 to 67
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2627 non-null   int64  
 1   Gender           2627 non-null   int64  
 2   Ever_Married     2627 non-null   float64
 3   Age              2627 non-null   int64  
 4   Graduated        2627 non-null   float64
 5   Profession       2627 non-null   float64
 6   Work_Experience  2627 non-null   float64
 7   Spending_Score   2627 non-null   int64  
 8   Family_Size      2627 non-null   float64
 9   Var_1            2627 non-null   float64
dtypes: float64(6), int64(4)
memory usage: 225.8 KB


In [40]:
df_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
4321,461253,1,0.0,18,0.0,4.0,0.0,1,5.0,4.0,D
5348,461312,0,0.0,18,0.0,4.0,3.0,1,3.0,2.0,D
6201,465354,0,0.0,18,0.0,3.0,0.0,1,5.0,2.0,C
1441,466846,0,0.0,18,0.0,4.0,2.0,1,4.0,2.0,D
4714,461279,0,0.0,18,0.0,3.0,4.0,1,2.0,6.0,A


In [41]:
df_train.sort_values(["ID"], axis=0, ascending=[True], inplace=True) 
df_test.sort_values(["ID"], axis=0, ascending=[True], inplace=True) 

In [42]:
y_train=df_train['Segmentation']

In [43]:
x_train=df_train.drop(['Segmentation'],axis=1)

In [44]:
#from sklearn import preprocessing

#x = x_train.values #returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
#x_scaled = min_max_scaler.fit_transform(x)
#x_train = pd.DataFrame(x_scaled)

In [45]:
#from sklearn import preprocessing

#x = df_test.values #returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
#x_scaled = min_max_scaler.fit_transform(x)
#df_test = pd.DataFrame(x_scaled)

In [46]:
y_train

7238    C
5546    C
4373    C
4695    D
3333    D
       ..
4791    C
570     D
5832    D
4140    A
4388    B
Name: Segmentation, Length: 8068, dtype: object

In [47]:
#map={"A":1,"B":2,"C":3,"D":4}
#y_train = y_train.map(map)


In [48]:
y_train

7238    C
5546    C
4373    C
4695    D
3333    D
       ..
4791    C
570     D
5832    D
4140    A
4388    B
Name: Segmentation, Length: 8068, dtype: object

In [49]:
class Class_Fit(object):
    def __init__(self, clf, params=None):
        if params:            
            self.clf = clf(**params)
        else:
            self.clf = clf()

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def grid_search(self, parameters, Kfold):
        self.grid = GridSearchCV(estimator = self.clf, param_grid = parameters, cv = Kfold)
        
    def grid_fit(self, X, Y):
        self.grid.fit(X, Y)
        
    def grid_predict(self, X, Y):
        self.predictions = self.grid.predict(X)
        print("Precision: {:.2f} % ".format(100*metrics.accuracy_score(Y, self.predictions)))

In [59]:

columns = ['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession', 'Spending_Score', 'Family_Size' , 'Var_1' ]
x = df_train[columns]
y = df_train['Segmentation']


In [60]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, train_size = 0.8)

In [61]:

gb = Class_Fit(clf = ensemble.GradientBoostingClassifier)
param_grid = {'n_estimators' : [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}
gb.grid_search(parameters = param_grid, Kfold = 5)
gb.grid_fit(X = x_train, Y = y_train)
gb.grid_predict(x_test, y_test)

Precision: 51.36 % 


In [62]:
g = plot_learning_curve(gb.grid.best_estimator_, "Gradient Boosting learning curves", X_train, Y_train,
                        ylim = [1.01, 0.7], cv = 5, 
                        train_sizes = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])

NameError: name 'plot_learning_curve' is not defined

In [ ]:
from xgboost import XGBClassifier

In [ ]:
XGBClassifier = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [ ]:
XGBClassifier.fit(x_train,y_train)

In [ ]:
y_pred=XGBClassifier.predict(df_test)


In [ ]:
y_pred

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1,leaf_size=10,weights='distance',p=1,n_jobs=-1, metric='minkowski',
                     metric_params=None)

In [ ]:
neigh.fit(x_train,y_train)

In [ ]:
y_pred=neigh.predict(df_test)

In [ ]:
np.savetxt("data115.csv", y_pred,delimiter=',',fmt=['%s'])

In [ ]:
#submission = pd.DataFrame({'ID':df_test['ID'], 'Segmentation':y_pred})

In [ ]:
x_train